In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, date
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage

%matplotlib inline

In [2]:
df = pd.read_csv('Reviews.csv')
print(df.shape)
df.head()

(71044, 25)


,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


In [3]:
df.columns

Index(['id', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'ean', 'keys',
       'manufacturer', 'manufacturerNumber', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title',
       'reviews.userCity', 'reviews.userProvince', 'reviews.username', 'upc'],
      dtype='object')

In [4]:
df.columns = ['PID', 'Brand', 'Categories', 'Added', 'Updated', 'EAN', 'keys',
       'Manufacturer', 'M_Number', 'P_Name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'Purchased',
       'Recommended', 'Rid', 'reviews.numHelpful',
       'Rating', 'R_URLs', 'Review', 'R_Title',
       'R_City', 'R_Province', 'R_Username', 'UPC']

In [5]:
df.head()

,PID,Brand,Categories,Added,Updated,EAN,keys,Manufacturer,M_Number,P_Name,...,Rid,reviews.numHelpful,Rating,R_URLs,Review,R_Title,R_City,R_Province,R_Username,UPC
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


In [6]:
df = df.drop(['Added', 'Updated', 'EAN', 'keys',
       'M_Number','reviews.date',
       'reviews.dateAdded', 'Manufacturer', 'reviews.dateSeen','Rid', 'reviews.numHelpful',
       'R_URLs','R_City', 'R_Province', 'R_Username', 'UPC'],axis =1).copy()
df.head()

,PID,Brand,Categories,P_Name,Purchased,Recommended,Rating,Review,R_Title
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Pink Friday: Roman Reloaded Re-Up (w/dvd),NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg Organic Cinnamon Toast Rice Cakes,True,NaN,5,Good flavor. This review was collected as part...,Good
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg Organic Cinnamon Toast Rice Cakes,True,NaN,5,Good flavor.,Good
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y Love Sensuality Pleasure Gel,False,False,1,I read through the reviews on here before look...,Disappointed
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y Love Sensuality Pleasure Gel,False,False,1,My husband bought this gel for us. The gel cau...,Irritation


In [7]:
df.columns

Index(['PID', 'Brand', 'Categories', 'P_Name', 'Purchased', 'Recommended',
       'Rating', 'Review', 'R_Title'],
      dtype='object')

In [8]:
df.isnull().sum()

PID                0
Brand              0
Categories         0
P_Name             0
Purchased      38886
Recommended    10615
Rating             0
Review            36
R_Title          476
dtype: int64

In [9]:
df = df.dropna(subset=['Review', 'R_Title']).copy()
df.head()

,PID,Brand,Categories,P_Name,Purchased,Recommended,Rating,Review,R_Title
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Pink Friday: Roman Reloaded Re-Up (w/dvd),NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg Organic Cinnamon Toast Rice Cakes,True,NaN,5,Good flavor. This review was collected as part...,Good
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg Organic Cinnamon Toast Rice Cakes,True,NaN,5,Good flavor.,Good
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y Love Sensuality Pleasure Gel,False,False,1,I read through the reviews on here before look...,Disappointed
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y Love Sensuality Pleasure Gel,False,False,1,My husband bought this gel for us. The gel cau...,Irritation


In [10]:
df.isnull().sum()

PID                0
Brand              0
Categories         0
P_Name             0
Purchased      38396
Recommended    10204
Rating             0
Review             0
R_Title            0
dtype: int64

In [11]:
df["Purchased"].fillna(method="backfill",inplace=True)
df.isnull().sum()

PID                0
Brand              0
Categories         0
P_Name             0
Purchased          2
Recommended    10204
Rating             0
Review             0
R_Title            0
dtype: int64

In [12]:
df["Recommended"].fillna(method="backfill",inplace=True)
df.isnull().sum()

PID            0
Brand          0
Categories     0
P_Name         0
Purchased      2
Recommended    0
Rating         0
Review         0
R_Title        0
dtype: int64

In [13]:
df= df.dropna(subset=["Purchased"]).copy()
df.isnull().sum()

PID            0
Brand          0
Categories     0
P_Name         0
Purchased      0
Recommended    0
Rating         0
Review         0
R_Title        0
dtype: int64

In [14]:
df.dtypes

PID            object
Brand          object
Categories     object
P_Name         object
Purchased      object
Recommended      bool
Rating          int64
Review         object
R_Title        object
dtype: object

In [15]:
df["Purchased"]=df["Purchased"].astype(bool)
df["Purchased"]=df["Purchased"].astype(int)
df["Recommended"]=df["Recommended"].astype(int)
df.dtypes

PID            object
Brand          object
Categories     object
P_Name         object
Purchased       int64
Recommended     int64
Rating          int64
Review         object
R_Title        object
dtype: object

In [16]:
df

,PID,Brand,Categories,P_Name,Purchased,Recommended,Rating,Review,R_Title
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Pink Friday: Roman Reloaded Re-Up (w/dvd),1,0,5,i love this album. it's very good. more to the...,Just Awesome
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg Organic Cinnamon Toast Rice Cakes,1,0,5,Good flavor. This review was collected as part...,Good
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg Organic Cinnamon Toast Rice Cakes,1,0,5,Good flavor.,Good
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y Love Sensuality Pleasure Gel,0,0,1,I read through the reviews on here before look...,Disappointed
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y Love Sensuality Pleasure Gel,0,0,1,My husband bought this gel for us. The gel cau...,Irritation
...,...,...,...,...,...,...,...,...,...
71037,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,Great item,Great
71038,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,Since I've starting wearing my contacts full-t...,Worth every penny.
71039,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,I'm dealing with dry eyes and very sensitive t...,Solves the problem
71040,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,Great stuff! Helps keep my contacts moist all ...,Great stuff!


In [17]:
# finding number of reviews per PID and then subsetting those wehre number of reviews is greater than 99
most=df[["PID"]].value_counts()
most=pd.DataFrame(most)
most.reset_index(level=0, inplace=True)
most.columns=["PID","N_Reviews"]
most=most[most["N_Reviews"]>99]
most.sort_values("PID",inplace=True,ascending=False)
most

,PID,N_Reviews
47,AVzRGDlw-jtxr-f3yfFH,223
25,AVzRE7KvGV-KLJ3aatQ9,562
34,AVqkGdQ6v8e3D1O-leAl,359
13,AVqVHr30nnc1JgDc3jVp,1412
37,AVq56bxmv8e3D1O-llXA,327
...,...,...
21,AVpe59io1cnluZ0-ZgDU,668
18,AVpe41TqilAPnD_xQH3d,757
44,AVpe31o71cnluZ0-YrSD,247
22,AV1l8zRZvKc47QAVhnAv,651


In [18]:
# most_df will be the most reviewed dataset where number of reviews will be more tha 99
most_df=df[df.PID.isin(most["PID"])].copy()
most_df.sort_values("PID",inplace=True,ascending=False)
most_df

,PID,Brand,Categories,P_Name,Purchased,Recommended,Rating,Review,R_Title
71041,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,Right price,My review
70930,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,1,1,5,I have been using this product for a month and...,I'm new to contacts
70899,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,This is the best and easiest to use cleaner th...,Great product
70898,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,1,1,5,My optician recommended this product after I c...,Extends life of soft contact lenses
70897,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,1,1,5,"Even though the package looks small, it really...",Great for Soft Contact Lenses
...,...,...,...,...,...,...,...,...,...
821,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,works well fast easy all over cleaning used in...,easy cleaning
820,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,I love your product but PLEASE BRING BACK THE ...,please bring back Mountain Berry scent!
819,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,Windex Glass Cleaner is so easy to use. Easy t...,Windex Original Glass Cleaner
818,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,It's handy and quickly clean up the dirty glas...,It's handy to keep it on hand to clean windshi...


In [19]:
conditions = [
    (most_df['Rating'] <= 2),
    (most_df['Rating'] > 2) & (most_df['Rating'] <= 4),
    (most_df['Rating'] > 4)
    ]
values= ["Frequent","Average","Low"]
most_df['Return_Type'] = np.select(conditions, values)
most_df

,PID,Brand,Categories,P_Name,Purchased,Recommended,Rating,Review,R_Title,Return_Type
71041,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,Right price,My review,Low
70930,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,1,1,5,I have been using this product for a month and...,I'm new to contacts,Low
70899,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,1,5,This is the best and easiest to use cleaner th...,Great product,Low
70898,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,1,1,5,My optician recommended this product after I c...,Extends life of soft contact lenses,Low
70897,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,1,1,5,"Even though the package looks small, it really...",Great for Soft Contact Lenses,Low
...,...,...,...,...,...,...,...,...,...,...
821,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,works well fast easy all over cleaning used in...,easy cleaning,Average
820,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,I love your product but PLEASE BRING BACK THE ...,please bring back Mountain Berry scent!,Average
819,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,Windex Glass Cleaner is so easy to use. Easy t...,Windex Original Glass Cleaner,Average
818,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,1,4,It's handy and quickly clean up the dirty glas...,It's handy to keep it on hand to clean windshi...,Average


In [20]:
# most_hated will be the dataset where the rating is less than 3
most_hated=most_df[most_df["Rating"]<3]

In [21]:
most_hated.reset_index(inplace=True)

In [22]:
most_hated.drop("index",axis=1,inplace=True)
most_hated

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,PID,Brand,Categories,P_Name,Purchased,Recommended,Rating,Review,R_Title,Return_Type
0,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,0,1,They reduced the bottle size but did not reduc...,"Value, declining",Frequent
1,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,0,1,Causes me to have Red eyes every time I use it...,Feels nice but causes Red eyes every time!,Frequent
2,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,0,1,I bought Opti-Free PureMoist and my eyes becam...,Red Eyes!,Frequent
3,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,0,1,Having used Opti-free replenish for years with...,Opti-free pure moist made my eyes sting,Frequent
4,AVzRGDlw-jtxr-f3yfFH,Opti-Free,"Personal Care,Eye & Ear Care,Eye Care,Contact ...",Opti-Free Puremoist Disinfecting Solution 10fl Oz,0,0,1,I switched to this solution from Opti-Free Rep...,Not for everyone - caused dryness and irritation,Frequent
...,...,...,...,...,...,...,...,...,...,...
4802,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,0,1,I just tried to clean normal interior film off...,"Very poor product, but was my go-to for years",Frequent
4803,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,0,1,We have been using Windex for years and re-sel...,Something has changed in the formula!,Frequent
4804,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,0,1,This product is no where near working successf...,Streaks and Film,Frequent
4805,AV1YGDqsGV-KLJ3adc-O,Windex,"Household Essentials,Cleaning Supplies,Glass C...",Windex Original Glass Cleaner Refill 67.6oz (2...,0,0,1,I used Windex Original on my glass top kitchen...,Windex Original no longer works,Frequent


In [23]:
import nltk
from nltk import word_tokenize
from nltk import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
# print(lem.lemmatize(“ate”))
# print(porter.stem(“ate”))

In [24]:
list=stopwords.words('english')

In [25]:
most_df["RR"]=[[lem.lemmatize(j) for j in set(i.split())] for i in most_df[["Review"]]["Review"] if i not in list]

In [26]:
most_df[["RR"]]

,RR
71041,"[price, Right]"
70930,"[with, every, month, about, right, have, lens,..."
70899,"[have, that, over, the, my, and, use, to, easi..."
70898,"[hooked!, should, month, after, distribute, My..."
70897,"[it, your, every, rid, have, lens, that, produ..."
...,...
821,"[well, fast, over, room, easy, review, wa, in,..."
820,"[it, your, PLEASE, poor, product, that, BRING,..."
819,"[it, cloth., the, window, though, Glass, to, W..."
818,"[problem, It's, solution, handy, the, neck, sa..."


In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def word_cloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
